# Выбор локации для скважины

[Введение](#beginning)

1. [Загрузка и подготовка данных](#preparing)


2. [Обучение и проверка модели](#modelling)

    2.1. [Модель для 1го региона](#model0)
    
    2.2. [Модель для 2го региона](#model1)
    
    2.3. [Модель для 3го региона](#model2)
    
    
3. [Подготовка к расчёту прибыли](#profit)


4. [Расчёт прибыли и рисков](#risk)


5. [Общий вывод](#finalizing)  

<a id="beginning"></a>
## Введение

В ходе данной работы будет построена модель для предсказания объёма запасов в новых скважинах, выбраны скважины с самыми высокими оценками значений и определён регион с максимальной суммарной прибылью отобранных скважин.

<a id="preparing"></a>
## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
try:
    data0 = pd.read_csv("C:/Users/Lantana/Documents/data_science/8_machine/geo_data_0.csv")
    data1 = pd.read_csv("C:/Users/Lantana/Documents/data_science/8_machine/geo_data_1.csv")
    data2 = pd.read_csv("C:/Users/Lantana/Documents/data_science/8_machine/geo_data_2.csv")

except:
    data0 = pd.read_csv('/datasets/geo_data_0.csv')
    data1 = pd.read_csv('/datasets/geo_data_1.csv')
    data2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data0.head()

,id,f0,f1,f2,product
0,txEyH,0.706,-0.498,1.221,105.280
1,2acmU,1.335,-0.340,4.365,73.038
2,409Wp,1.023,0.152,1.420,85.266
3,iJLyR,-0.032,0.139,2.979,168.621
4,Xdl7t,1.988,0.155,4.752,154.037


In [4]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data0.describe()

,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,0.500,0.250,2.503,92.500
std,0.872,0.504,3.248,44.289
min,-1.409,-0.848,-12.088,0.000
25%,-0.073,-0.201,0.288,56.498
50%,0.502,0.250,2.516,91.850
75%,1.074,0.701,4.715,128.564
max,2.362,1.344,16.004,185.364


In [6]:
data1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001,-8.276,-0.006,3.179
1,62mP7,14.272,-3.475,0.999,26.953
2,vyE1P,6.263,-5.948,5.001,134.766
3,KcrkZ,-13.081,-11.506,4.999,137.945
4,AHL4O,12.702,-8.147,5.004,134.766


In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data1.describe()

,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,1.141,-4.797,2.495,68.825
std,8.966,5.120,1.704,45.944
min,-31.610,-26.359,-0.018,0.000
25%,-6.299,-8.268,1.000,26.953
50%,1.153,-4.813,2.011,57.086
75%,8.621,-1.333,4.000,107.813
max,29.422,18.734,5.020,137.945


In [9]:
data2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.147,0.963,-0.829,27.759
1,WJtFt,0.263,0.270,-2.530,56.070
2,ovLUW,0.195,0.289,-5.586,62.872
3,q6cA6,2.236,-0.554,0.930,114.573
4,WPMUX,-0.516,1.716,5.899,149.601


In [10]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
data2.describe()

,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,0.002,-0.002,2.495,95.000
std,1.732,1.730,3.473,44.750
min,-8.760,-7.084,-11.970,0.000
25%,-1.162,-1.175,0.130,59.450
50%,0.009,-0.009,2.484,94.926
75%,1.159,1.164,4.859,130.595
max,7.238,7.845,16.739,190.030


In [12]:
print(data0.corr())

            f0     f1     f2  product
f0       1.000 -0.441 -0.003    0.144
f1      -0.441  1.000  0.002   -0.192
f2      -0.003  0.002  1.000    0.484
product  0.144 -0.192  0.484    1.000


In [13]:
print(data1.corr())

            f0     f1     f2  product
f0       1.000  0.182 -0.002   -0.030
f1       0.182  1.000 -0.003   -0.010
f2      -0.002 -0.003  1.000    0.999
product -0.030 -0.010  0.999    1.000


In [14]:
print(data2.corr())

            f0     f1     f2  product
f0       1.000  0.001 -0.000   -0.002
f1       0.001  1.000  0.001   -0.001
f2      -0.000  0.001  1.000    0.446
product -0.002 -0.001  0.446    1.000


***Вывод:***
Данные готовы к обучению: в данных нет пропусков, некорректных значений, мультиколлинеарность признаков отсутствует.

<a id="preparing"></a>
## Обучение и проверка модели

1. Разделим данные по всем регионам на обучающую (train) и валидационную (valid) выборки в соотношении 75% - 25%. И извлечём признаки для обучения: целевой - target, остальные - features.

2. Обучим модели линейной регрессии для каждого региона. И сделаем предсказания на валидационных выборках.

3. Для каждого регионаа напечатаем средний запас предсказанного сырья, RMSE модели, R2 модели.

<a id="model0"></a>
### Модель для 1го региона

In [15]:
features0 = data0.drop(['product', 'id'], axis=1)
target0 = data0['product']
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

In [16]:
model0 = LinearRegression()
model0.fit(features_train0, target_train0)
predicted_valid0 = model0.predict(features_valid0)
result0 = mean_squared_error(target_valid0, predicted_valid0) ** 0.5
print("RMSE модели для первого региона:", result0)
print("R2 для 1го региона =", r2_score(target_valid0, predicted_valid0))
mean_predicted_valid_0 = predicted_valid0.mean()
print("Средний запас предсказанного сырья для 1го региона:", mean_predicted_valid_0)
mean_target0 = data0['product'].mean()
print(mean_target0)

RMSE модели для первого региона: 37.5794217150813
R2 для 1го региона = 0.27994321524487786
Средний запас предсказанного сырья для 1го региона: 92.59256778438038
92.50000000000001


<a id="model1"></a>
### Модель для 2го региона

In [17]:
features1 = data1.drop(['product', 'id'], axis=1)
target1 = data1['product']
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

In [18]:
model1 = LinearRegression()
model1.fit(features_train1, target_train1)
predicted_valid1 = model1.predict(features_valid1)
result1 = mean_squared_error(target_valid1, predicted_valid1) ** 0.5
print("RMSE модели для второго региона:", result1)
print("R2 для 2го региона =", r2_score(target_valid1, predicted_valid1))
mean_predicted_valid_1 = predicted_valid1.mean()
print("Средний запас предсказанного сырья для 2го региона:", mean_predicted_valid_1)
mean_target1 = data1['product'].mean()
print(mean_target1)

RMSE модели для второго региона: 0.8930992867756155
R2 для 2го региона = 0.9996233978805127
Средний запас предсказанного сырья для 2го региона: 68.72854689544602
68.82500000000002


<a id="model2"></a>
### Модель для 3го региона

In [19]:
features2 = data2.drop(['product', 'id'], axis=1)
target2 = data2['product']
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

In [20]:
model2= LinearRegression()
model2.fit(features_train2, target_train2)
predicted_valid2 = model2.predict(features_valid2)
result2 = mean_squared_error(target_valid2, predicted_valid2) ** 0.5
print("RMSE модели для третьего региона:", result2)
print("R2 для 3го региона =", r2_score(target_valid2, predicted_valid2))
mean_predicted_valid_2 = predicted_valid2.mean()
print("Средний запас предсказанного сырья для 3го региона:", mean_predicted_valid_2)
mean_target2 = data2['product'].mean()
print(mean_target2)

RMSE модели для третьего региона: 40.02970873393434
R2 для 3го региона = 0.20524758386040443
Средний запас предсказанного сырья для 3го региона: 94.96504596800489
95.00000000000004


***Вывод:***

In [21]:
summary = pd.DataFrame({'region': ['first', 'second', 'third'], 'average_product': [92.59, 68.73, 94.97],  'rmse': [37.58, 0.89, 40.03],  'r2': [0.28, 0.99, 0.21]})
summary

,region,average_product,rmse,r2
0,first,92.590,37.580,0.280
1,second,68.730,0.890,0.990
2,third,94.970,40.030,0.210


Во втором регионе мы получили мЕньшее значение среднего запаса предсказанного сырья, однако модель для него имеет наивысшее качество - R2 почти 1, т.к. ранее мы выяснили, что признак f2 на 99% указывает на объём запаса в скважине.
В первом и третьем регионах качество модели низкое, R2 чуть более 20%, отклонение RMSE почти 40 тыс. баррелей. Однако среднее значение запаса предсказанного сырья выше = 92,5 и 95 тыс. баррелей соответственно.

<a id="profit"></a>
## Подготовка к расчёту прибыли

Подготовимся к расчёту прибыли: сохраним все ключевые для расчётов значения (бюджет на разработку скважин и доход с единицы продукта) в отдельных переменных (budget и revenue_pro_1000_barrel)

In [22]:
budget = 10000000000
print('Бюджет на разработку скважин в регионе, руб.: {:_.0f}'.format(budget))
revenue_pro_1000_barrel = 450000
print('Доход с единицы продукта, руб.: {:_.0f}'.format(revenue_pro_1000_barrel))


Бюджет на разработку скважин в регионе, руб.: 10_000_000_000
Доход с единицы продукта, руб.: 450_000


Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины:

In [23]:
breakeven_barrel = (budget/revenue_pro_1000_barrel)/200
print('Достаточный объём сырья для безубыточной разработки новой скважины, тыс. барр: {:_.2f}'.format(breakeven_barrel))

Достаточный объём сырья для безубыточной разработки новой скважины, тыс. барр: 111.11


***Вывод:***

In [24]:
summary

,region,average_product,rmse,r2
0,first,92.590,37.580,0.280
1,second,68.730,0.890,0.990
2,third,94.970,40.030,0.210


Достаточный объём сырья для безубыточной разработки новой скважины (111 тыс. баррелей) выше, чем средний запас предсказанного сырья по всем трём регионам (максимально полученный - 95 тыс. баррелей для третьего региона). Т.е. нет такого региона, где можно не глядя разрабатывать скважины и не получить убытка.

<a id="risk"></a>
## Расчёт прибыли и рисков 

Для удобства расчётов создадим новые таблицы с фактическим и предсказанным объёмом запасов в скважинах (для каждого региона).

In [25]:
data0_predict = pd.DataFrame()
data0_predict['product'] = target_valid0
data0_predict['predict'] = predicted_valid0

data0_predict.head()

,product,predict
71751,10.039,95.895
80493,114.551,77.573
2655,132.604,77.893
53233,169.072,90.175
91141,122.325,70.510


In [26]:
data1_predict = pd.DataFrame()
data1_predict['product'] = target_valid1
data1_predict['predict'] = predicted_valid1

data1_predict.head()

,product,predict
71751,80.860,82.663
80493,53.907,54.432
2655,30.132,29.749
53233,53.907,53.552
91141,0.000,1.244


In [27]:
data2_predict = pd.DataFrame()
data2_predict['product'] = target_valid2
data2_predict['predict'] = predicted_valid2

data2_predict.head()

,product,predict
71751,61.212,93.600
80493,41.850,75.105
2655,57.777,90.067
53233,100.054,105.162
91141,109.897,115.303


Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели, которая будет выбирать 200 скважин с максимальными значениями предсказаний, а затем суммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
Затем рассчитаем прибыль для каждого региона.

In [28]:
def profit_func(product, predict):
    predictions_sorted = predict.sort_values(ascending=False)
    product_chosen = product[predictions_sorted.index][:200]
    profit = product_chosen.sum()*revenue_pro_1000_barrel - budget
    return profit

In [29]:
profit0 = profit_func(data0_predict['product'], data0_predict['predict'])
print('Прибыль для первого региона, руб.: {:_.0f}'.format(profit0))

Прибыль для первого региона, руб.: 3_320_826_043


In [30]:
profit1 = profit_func(data1_predict['product'], data1_predict['predict'])
print('Прибыль для второго региона, руб.: {:_.0f}'.format(profit1))

Прибыль для второго региона, руб.: 2_415_086_697


In [31]:
profit2 = profit_func(data2_predict['product'], data2_predict['predict'])
print('Прибыль для третьего региона, руб.: {:_.0f}'.format(profit2))

Прибыль для третьего региона, руб.: 2_710_349_964


Посчитаем риски и прибыль для каждого региона.
Для этого применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Методом sample исследуются 500 точек, из которых потом в функции прибыли будут выбраны 200 лучших.
Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. 

In [32]:
state = np.random.RandomState(12345)
values0 = []
for i in range(1000):
    data0_predict_subsample = data0_predict.sample(500, replace=True, random_state=state)
    values0.append(profit_func(data0_predict_subsample['product'], data0_predict_subsample['predict']))

values0 = pd.Series(values0)
    
lower0 = values0.quantile(0.025)
upper0 = values0.quantile(0.975)
mean0 = values0.mean()
risk0 = st.percentileofscore(values0, 0)
print('Средняя прибыль для первого региона, руб.: {:_.0f}'.format(mean0))
print('95%-ый доверительный интервал от: {:_.0f}'.format(lower0))
print('до: {:_.0f}'.format(upper0))
print("Вероятность убытков для 1го региона, %:", risk0)

Средняя прибыль для первого региона, руб.: 425_938_527
95%-ый доверительный интервал от: -102_090_095
до: 947_976_353
Вероятность убытков для 1го региона, %: 6.0


In [33]:
state = np.random.RandomState(12345)
values1 = []
for i in range(1000):
    data1_predict_subsample = data1_predict.sample(500, replace=True, random_state=state)
    values1.append(profit_func(data1_predict_subsample['product'], data1_predict_subsample['predict']))

values1 = pd.Series(values1)
    
lower1 = values1.quantile(0.025)
upper1 = values1.quantile(0.975)
mean1 = values1.mean()
risk1 = st.percentileofscore(values1, 0)

print('Средняя прибыль для второго региона, руб.: {:_.0f}'.format(mean1))
print('95%-ый доверительный интервал от: {:_.0f}'.format(lower1))
print('до: {:_.0f}'.format(upper1))
print("Вероятность убытков для 2го региона, %:", risk1)

Средняя прибыль для второго региона, руб.: 515_222_773
95%-ый доверительный интервал от: 68_873_225
до: 931_547_591
Вероятность убытков для 2го региона, %: 1.0


In [34]:
state = np.random.RandomState(12345)
values2 = []
for i in range(1000):
    data2_predict_subsample = data2_predict.sample(500, replace=True, random_state=state)
    values2.append(profit_func(data2_predict_subsample['product'], data2_predict_subsample['predict']))

values2 = pd.Series(values2)
    
lower2 = values2.quantile(0.025)
upper2 = values2.quantile(0.975)
mean2 = values2.mean()
risk2 = st.percentileofscore(values2, 0)
print('Средняя прибыль для третьего региона, руб.: {:_.0f}'.format(mean2))
print('95%-ый доверительный интервал от: {:_.0f}'.format(lower2))
print('до: {:_.0f}'.format(upper2))
print("Вероятность убытков для 3го региона, %:", risk2)

Средняя прибыль для третьего региона, руб.: 435_008_363
95%-ый доверительный интервал от: -128_880_547
до: 969_706_954
Вероятность убытков для 3го региона, %: 6.4


***Вывод:***

По условиям задачи мы отбрасываем регионы с вероятностью убытков более 2,5% (в нашем случае - первый и третий). Таким образом для разработки скважин предлагаем второй регион. Для него же обеспечивается наибольшая средняя прибыль - более 500 млн руб.

<a id="finalizing"></a>
## Общий вывод
В ходе работы были проанализированы данные геологоразведки по трём регионам, построены модели для предсказания объёма запасов в новых скважинах, выбраны скважины с самыми высокими оценками значений и предложен регион с максимальной суммарной прибылью отобранных скважин.

- Во втором регионе мы получили мЕньшее значение среднего запаса предсказанного сырья, однако модель для него имеет наивысшее качество - R2 почти 1, т.к. ранее мы выяснили, что признак f2 на 99% указывает на объём запаса в скважине. В первом и третьем регионах качество модели низкое, R2 чуть более 20%, отклонение RMSE почти 40 тыс. баррелей. Однако среднее значение запаса предсказанного сырья выше = 92,5 и 95 тыс. баррелей соответственно.

- Достаточный объём сырья для безубыточной разработки новой скважины (111 тыс. баррелей) выше, чем средний запас предсказанного сырья по всем трём регионам (максимально полученный - 95 тыс. баррелей для третьего региона). Т.е. нет такого региона, где можно не глядя разрабатывать скважины и не получить убытка.

- По условиям задачи были отброшены регионы с вероятностью убытков более 2,5% (в нашем случае - первый и третий). Таким образом для разработки скважин был предложен второй регион. Для него же обеспечивается наибольшая средняя прибыль - более 500 млн руб.